# Unit testing in Jupyter notebooks with ipytest: Multiply tests

## Table of contents
1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Tests](#tests)
3. [Technical Issues](#technical-issues)

## Introduction <a name="introduction"></a>

**Unit tests** in **Jupyter notebooks** can be run using the excellent [ipytest](https://github.com/chmp/ipytest) package. It supports both [pytest](https://docs.pytest.org/en/latest/) and [unittest](https://docs.python.org/3/library/unittest.html).

This *notebook* shows examples of writing tests with both frameworks. A simple example of multiplying two ints or floats is used.

To run the tests in this *notebook*, the functions they are testing in [multiply.ipynb](../notebooks/multiply.ipynb) will need to loaded into the same [IPython](https://ipython.readthedocs.io/en/stable/) interactive namespace. The simplest way to do this during development is to use the excellent [JupyterLab](https://github.com/jupyterlab/jupyterlab) computational environment to connect both notebooks to the same [kernel](http://jupyter.readthedocs.io/en/latest/architecture/how_jupyter_ipython_work.html#the-ipython-kernel). This can be achieved through the *Kernel > Change Kernel* option in the JupyterLab user interface. Please see the [JupyterLab documentation](http://jupyterlab.readthedocs.io/en/stable/) for more information on [managing kernels](http://jupyterlab.readthedocs.io/en/stable/user/running.html).

## Setup <a name="setup"></a>

In [3]:
import unittest

import ipytest
import numpy.testing as npt
import pytest

Specify the following options to pass to pytest:

* Verbose output
* Colored output

For more information see [pytest options](http://pytest.readthedocs.io/en/reorganize-docs/new-docs/user/commandlineuseful.html). 

In [4]:
PYTEST_OPTIONS = ['-v', '--color=yes']

Change the following constant to True to specify to run the tests with *unittest* instead of *pytest*. Unittest is very basic and has less options than pytest. **The output is not pretty** and it has **verbose syntax**. I suppose it does have the advantage of shipping with the python standard library, but I still prefer *pytest*.

In [5]:
UNIT_TEST = False

Load some useful utility functions:

In [6]:
# %load main_module.py
"""This file must be loaded into a Jupyter notebook (using the `%load` magic) and executed there to work
   correctly.
   Note: 
       If it is placed into a module and imported into the notebook then `__name__` will be the module
       name and `__file__` will be in the `globals` object.
"""

def is_main_module():
    """Returns whether this notebook is the main module (i.e. not being run from another notebook).Taken from: 
       https://blog.sicara.com/present-data-science-results-jupyter-notebook-import-into-another-108433bc8505
    Returns:
        True if this notebook is the main module, False otherwise.
    """
    return __name__ == '__main__' and '__file__' not in globals()

If this *notebook* is being run from another *notebook* (which it shouldn't be) or program (i.e. a *test runner* such as *pytest* or *unittest*) then "run" the implementation notebook. This is done for import purposes. More to come on this later.

In [7]:
if not is_main_module():
    %run ../notebooks/multiply.ipynb

## Tests <a name="tests"></a>

It is possible to create test cases at the global level using fixtures to test functions inside of classes. It is also possible to parametrize tests.

In [8]:
if is_main_module():
    ipytest.clean_tests(pattern='test_*')
    ipytest.clean_tests(pattern='Test*')

@pytest.fixture
def multiplier():
    return Multiply()

@pytest.mark.parametrize('a,b,expected', [
    (2, 2, 4),
    (-1, 3, -3),
    (0, 5, 0),
])
def test_multiply(multiplier, a, b, expected):
    multiplier.multiply(a, b) == expected
    
if is_main_module():
    if UNIT_TEST:
        ipytest.run_tests(doctest=True, items={'test_multiply': globals()['test_multiply'],
                                               'multiply': globals()['multiply']});
    else:
        ipytest.run_pytest(filename='test_multiply.ipynb', pytest_options=PYTEST_OPTIONS);

============================= test session starts =============================
platform win32 -- Python 3.6.4, pytest-3.5.1, py-1.5.3, pluggy-0.6.0 -- C:\Users\covuworie\.virtualenvs\notebook-testing-9WWLULs9\Scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\covuworie\D-Drive\notebook-testing\tests, inifile:
collecting ... collected 3 items

test_multiply.py::test_multiply[2-2-4] <- <ipython-input-8-22f0b976eb05> PASSED [ 33%]
test_multiply.py::test_multiply[-1-3--3] <- <ipython-input-8-22f0b976eb05> PASSED [ 66%]
test_multiply.py::test_multiply[0-5-0] <- <ipython-input-8-22f0b976eb05> PASSED [100%]

========================== 3 passed in 0.07 seconds ===========================


In [9]:
if is_main_module():
    ipytest.clean_tests(pattern='test_*')
    ipytest.clean_tests(pattern='Test*')

def test_multiply_floats():
    npt.assert_allclose(multiply_floats(0.1, 0.2), 0.02)

if is_main_module():
    if UNIT_TEST:
        ipytest.run_tests(doctest=False, 
                          items={'test_multiply_floats': globals()['test_multiply_floats']})
    else:
        ipytest.run_pytest(filename='test_multiply.ipynb', pytest_options=PYTEST_OPTIONS);

============================= test session starts =============================
platform win32 -- Python 3.6.4, pytest-3.5.1, py-1.5.3, pluggy-0.6.0 -- C:\Users\covuworie\.virtualenvs\notebook-testing-9WWLULs9\Scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\covuworie\D-Drive\notebook-testing\tests, inifile:
collecting ... collected 1 item

test_multiply.py::test_multiply_floats <- <ipython-input-9-3972b686661a> PASSED [100%]

========================== 1 passed in 0.03 seconds ===========================


## Technical Issues <a name="technical-issues"></a>

Calls to `ipytest.run_tests()` invoke `unittest.TextTestRunner()`. However, calls to `ipytest.run_pytest()` actually invoke `pytest.main()`. It is advised to only make one call to `pytest.main()` (or any other `main()` function in python) from within a process, due to [python's import caching mechanism](https://github.com/pytest-dev/pytest/issues/3143). How come I have made multiple calls to this function? 

A notebook is not a python module as it is not imported. Therefore it is perfectly OK to make multiple calls to the main function of *pytest* (or any other ` main()` function in python).